In [ ]:
%load_ext autoreload
%autoreload 2
from MCTS import MCTS

In [ ]:
import time
from IPython.display import clear_output
import os
from stockfish import Stockfish


#path = os.getcwd()+"\stockfish_15.1_win_x64_avx2\stockfish-windows-2022-x86-64-avx2.exe"
#stockfish = Stockfish(path=path)
#stockfish.set_elo_rating(1150)

In [47]:
import chess
board = chess.Board()
time_limit = 15

for i in range(50):
    """
    stockfish.set_fen_position(board.fen())
    board.push_san(stockfish.get_best_move())
    clear_output(wait=True)
    display(board)
    time.sleep(0.5)
    """
    if board.turn:
        search = MCTS(board, time_limit=10, num_simulations=10)
        searchy.search()
        result = search.best_move()
    else:
        result = random.choice(list(board.legal_moves))
    board.push(result)
    clear_output(wait=True)
    display(board)
    time.sleep(0.1)

TypeError: heap argument must be a list